In [1]:
import numpy as np
import matplotlib.pyplot as plt

In [2]:
"""
activation functions and their derivatives
"""

def relu_kth(x, k):
    return np.maximum(x**k, 0)

def relu(x):
    return np.maximum(x, 0)

def d_relu(x):
    return 1.0 * (x > 0)

def d_relu_kth(x, k):
    return k * x**(k-1) * (x > 0)

In [3]:
"""
analytic solution: Equation (4)-(5) on page 3

u = <x, x'> / ||x|| ||x'||

k(u) = u k_0(u) + k_1(u)

k_0 = 1/pi (pi - arccos(u))
k_1 = 1/pi (u (pi - arccos(u)) + sqrt(1 - u^2))

assuming x == x'
"""

def kappa(u):
    pi = np.pi
    k_0 = (1/pi) * (pi - np.arccos(u))
    k_1 = (1/pi) * (u * (pi - np.arccos(u)) + np.sqrt(1 - u**2))
    return u * k_0 + k_1

def analytic_kernel_entry_2d(x, y): # x, y vector
    # don't need norm_prod because x y are both unit vector, norm=1
    norm_prod = np.linalg.norm(x) * np.linalg.norm(y)
    # for 1d input on line, it's just x * y
    inner_prod = x.dot(y)
    # for numeircal stability: if not -1 < u < 1, arccos(u) is undefined
    if (inner_prod > 1.0):
        inner_prod = 1.0
    if (inner_prod < -1.0):
        inner_prod = -1.0
    u = inner_prod
    
    return kappa(u)

In [44]:
"""
Numerical approach at
https://papers.nips.cc/paper/2019/file/c4ef9c39b300931b69a36fb3dbb8d60e-Paper.pdf
equation (3) on page 3 and section 3.3 on page 7 
"""

# K(x, x') = <x, x'> E[sig'(<w, x>)sig'(<w, x'>)] + E[sig(<w, x>)sig(<w, x'>)]
# E {w ~ N(0, 1)}

def numerical_kernel_entry(x, y, num_samples=1000):

    w1, w2 = np.random.normal(0.0, 1.0, size=[2, num_samples])
    
    w1_ = np.tile(w1, [2, 1]).T
    w2_ = np.tile(w2, [2, 1]).T   
    
    print(w1_)
    x_ = np.tile(x, (num_samples, 1))
    print(x_)
    y_ = np.tile(y, (num_samples, 1))
    
    w1x = w1_ * x_
    w1y = w1_ * y_
    sigma_w1x = relu(w1x)
    sigma_w1y = relu(w1y)

    w2x = w2_ * x_
    w2y = w2_ * y_
    sigma_w2x = d_relu(w2x)
    sigma_w2y = d_relu(w2y)

    expection_1 = sigma_w1x.dot(sigma_w1y) / num_samples
    expection_2 = sigma_w2x.dot(sigma_w2y) / num_samples
    
    return (x.dot(y)) * expection_1 + expection_2

In [45]:
# execute
def calc_NTK_2d(kernel_entry_func):
    num_inputs = 100
    pi = np.pi
    # sin(theta) and cos(theta), theta ~ (0, 2*pi) uniform\
    # take 100 points on uniit circle
    theta = np.linspace(0.0, 2 * pi, num=num_inputs)
    x = np.asarray((np.cos(theta), np.sin(theta)))

    # make kernel
    kernel = np.zeros((num_inputs, num_inputs))

    for i in range(num_inputs):
        for j in range(num_inputs):
            kernel[i][j] = kernel_entry_func(x[:, i], x[:, j])
    
    # sort eigenvalues in descending order
    eigenvalues = np.linalg.eigvals(kernel)
    sorted_eigenvalues = np.sort(eigenvalues)[::-1] 
    # plt.plot(sorted_eigenvalues[20:])
    plt.loglog(sorted_eigenvalues[:-1])

In [46]:
calc_NTK_2d(numerical_kernel_entry)

[[-0.98672335 -0.98672335]
 [-1.89843487 -1.89843487]
 [ 0.94684495  0.94684495]
 ...
 [-0.13421126 -0.13421126]
 [ 0.21705815  0.21705815]
 [ 0.45207803  0.45207803]]
[[1. 0.]
 [1. 0.]
 [1. 0.]
 ...
 [1. 0.]
 [1. 0.]
 [1. 0.]]


ValueError: shapes (1000,2) and (1000,2) not aligned: 2 (dim 1) != 1000 (dim 0)

In [7]:
a = np.arange(5)

In [17]:
a

array([0, 1, 2, 3, 4])

In [25]:
np.tile(a, [3, 1]).T

array([[0, 0, 0],
       [1, 1, 1],
       [2, 2, 2],
       [3, 3, 3],
       [4, 4, 4]])